In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Step 1: Load the full dataset with all columns
df = pd.read_excel('final_merged_with_NULLS.xlsx')

# Step 2: Rename the HDI column for convenience
df = df.rename(columns={"Human Development Index (value)": "HDI_Score"})

# Step 3: Sort by Country and Year
df = df.sort_values(by=["Country", "Year"])

# Step 4: Prepare a DataFrame to hold the forecast results
forecast_results = pd.DataFrame()

# Step 5: Forecast HDI scores for the next 10 years for each country
forecast_steps = 10

for country, country_df in df.groupby("Country"):
    # Drop rows with missing HDI values before fitting the model
    country_df_non_null = country_df.dropna(subset=["HDI_Score"])
    
    # Fit an ARIMA model on the HDI time series for the current country
    if not country_df_non_null.empty:
        model = ARIMA(country_df_non_null["HDI_Score"], order=(1, 1, 1))
        fitted_model = model.fit()

        # Forecast the HDI scores for the next 10 years
        forecast = fitted_model.forecast(steps=forecast_steps)

        # Generate the years for the forecast
        last_year = country_df_non_null["Year"].max()
        forecast_years = np.arange(last_year + 1, last_year + forecast_steps + 1)

        # Compile forecast data into a DataFrame for the current country
        country_forecast_df = pd.DataFrame({
            "Country": country,
            "Year": forecast_years,
            "Predicted_HDI_Score": forecast
        })

        # Append the country's forecast data to the main forecast results DataFrame
        forecast_results = pd.concat([forecast_results, country_forecast_df], ignore_index=True)

# Step 6: Merge the forecasted data back with the original DataFrame, preserving all rows and columns
final_df = pd.merge(df, forecast_results, on=["Country", "Year"], how="outer")

# Step 7: Show Final DF + save as CSV
print("Final Data with Forecasted HDI Scores:")
print(final_df)

output_csv_path = "time_series_data.csv"
final_df.to_csv(output_csv_path, index=False)

Final Data with Forecasted HDI Scores:
          Country      Region IncomeGroup  Year  HDI_Score  HDI Rank  \
0     Afghanistan  South Asia  Low income  1990      0.284       0.0   
1     Afghanistan  South Asia  Low income  1991      0.292       0.0   
2     Afghanistan  South Asia  Low income  1992      0.299       0.0   
3     Afghanistan  South Asia  Low income  1993      0.307       0.0   
4     Afghanistan  South Asia  Low income  1994      0.300       0.0   
...           ...         ...         ...   ...        ...       ...   
8380     Zimbabwe         NaN         NaN  2028        NaN       NaN   
8381     Zimbabwe         NaN         NaN  2029        NaN       NaN   
8382     Zimbabwe         NaN         NaN  2030        NaN       NaN   
8383     Zimbabwe         NaN         NaN  2031        NaN       NaN   
8384     Zimbabwe         NaN         NaN  2032        NaN       NaN   

      Expected Years of Schooling (years)  Mean Years of Schooling (years)  \
0                 

In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings("ignore")

# Step 1: Load the full dataset with all columns
df = pd.read_excel('final_merged_with_NULLS.xlsx')

# Step 2: Rename the HDI column for convenience
df = df.rename(columns={"Human Development Index (value)": "HDI_Score"})

# Step 3: Sort by Country and Year
df = df.sort_values(by=["Country", "Year"])

# Step 4: Prepare a DataFrame to hold the forecast results
forecast_results = pd.DataFrame()

# Step 5: Forecast HDI scores for the next 10 years for each country
forecast_steps = 10
metrics_results = []

for country, country_df in df.groupby("Country"):
    # Drop rows with missing HDI values before fitting the model
    country_df_non_null = country_df.dropna(subset=["HDI_Score"])
    
    # Fit an ARIMA model on the HDI time series for the current country
    if not country_df_non_null.empty:
        # Prepare training and testing data
        train_data = country_df_non_null["HDI_Score"].values
        train_size = int(len(train_data) * 0.8)
        train, test = train_data[:train_size], train_data[train_size:]

        # Fit the ARIMA model (you can modify the order based on your dataset)
        model = ARIMA(train, order=(1, 1, 1))
        fitted_model = model.fit()

        # Adjust forecast_steps based on the available test data length
        forecast_steps = min(len(test), 10)  # Set the forecast steps to the minimum of test data length or 10

        # Forecast the HDI scores for the available steps
        forecast = fitted_model.forecast(steps=forecast_steps)

        # Now both forecast and actual_test_values should have the same length
        actual_test_values = test[-forecast_steps:]  # Use only the available test values

        # RMSE (Root Mean Squared Error)
        rmse = np.sqrt(mean_squared_error(actual_test_values, forecast))

        # MAE (Mean Absolute Error)
        mae = mean_absolute_error(actual_test_values, forecast)

        # MAPE (Mean Absolute Percentage Error)
        mape = np.mean(np.abs((actual_test_values - forecast) / actual_test_values)) * 100

        # R² (Coefficient of Determination)
        ss_res = np.sum((actual_test_values - forecast) ** 2)
        ss_tot = np.sum((actual_test_values - np.mean(actual_test_values)) ** 2)
        r2 = 1 - (ss_res / ss_tot)

        # AIC (Akaike Information Criterion) and BIC (Bayesian Information Criterion)
        aic = fitted_model.aic
        bic = fitted_model.bic

        # Store the metrics for the current country
        metrics_results.append({
            "Country": country,
            "RMSE": rmse,
            "MAE": mae,
            "MAPE": mape,
            "R²": r2,
            "AIC": aic,
            "BIC": bic
        })

        # Generate the years for the forecast
        last_year = country_df_non_null["Year"].max()
        forecast_years = np.arange(last_year + 1, last_year + forecast_steps + 1)

        # Compile forecast data into a DataFrame for the current country
        country_forecast_df = pd.DataFrame({
            "Country": country,
            "Year": forecast_years,
            "Predicted_HDI_Score": forecast
        })

        # Append the country's forecast data to the main forecast results DataFrame
        forecast_results = pd.concat([forecast_results, country_forecast_df], ignore_index=True)

# Step 6: Convert the metrics results into a DataFrame
metrics_df = pd.DataFrame(metrics_results)

# Step 7: Show Final DF with metrics and save as CSV
print("Model Evaluation Metrics for ARIMA:")
print(metrics_df)
metrics_df.to_csv("arima_model_metrics.csv", index=False)

Model Evaluation Metrics for ARIMA:
                                Country      RMSE       MAE       MAPE  \
0                           Afghanistan  0.027030  0.019893   4.203652   
1                               Albania  0.012601  0.010069   1.277718   
2                               Algeria  0.012880  0.010854   1.471064   
3                               Andorra  0.011701  0.008765   1.012728   
4                                Angola  0.059655  0.051730   8.719631   
..                                  ...       ...       ...        ...   
190  Venezuela (Bolivarian Republic of)  0.065660  0.057215   8.109577   
191                            Viet Nam  0.005307  0.003966   0.550512   
192                               Yemen  0.098941  0.090399  21.124175   
193                              Zambia  0.014566  0.012156   2.139503   
194                            Zimbabwe  0.008554  0.006707   1.213460   

             R²         AIC         BIC  
0     -6.998255 -172.075498 -168.